In [40]:
import ipywidgets as widgets
from ipyleaflet import Map, WidgetControl, GeoJSON, Marker, LayerException
import geopandas as gpd
import json

In [41]:
# Path to your shapefile
shapefile_path = './county_boundary/cb_2018_us_county_5m.shp'

# Reading the shapefile
gdf = gpd.read_file(shapefile_path)
projected_gdf = gdf.to_crs(epsg=2163)



In [ ]:
state_fips_mapping = {
    '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA',
    '08': 'CO', '09': 'CT', '10': 'DE', '12': 'FL', '13': 'GA',
    '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA',
    '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD',
    '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO',
    '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ',
    '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH',
    '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC',
    '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT',
    '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI', '56': 'WY'
}

In [ ]:
# Update state names in the GeoDataFrame using the mapping
gdf['StateName'] = gdf['STATEFP'].apply(lambda x: state_fips_mapping.get(x, ''))

# Dropdown for states
state_dropdown = widgets.Dropdown(
    options=[''] + sorted(gdf['StateName'].unique()),
    description='State:',
    disabled=False,
)

# Dropdown for counties (initially empty)
county_dropdown = widgets.Dropdown(
    description='County:',
    disabled=True,  # Disabled until a state is selected
)

def update_county_dropdown(*args):
    selected_state = state_dropdown.value
    if selected_state:
        filtered_gdf = gdf[gdf['StateName'] == selected_state]
        county_dropdown.options = [''] + sorted(filtered_gdf['NAME'].unique())
        county_dropdown.disabled = False
    else:
        county_dropdown.options = []
        county_dropdown.disabled = True

state_dropdown.observe(update_county_dropdown, 'value')

# Initialize the map
mymap = Map(center=(37.0902, -95.7129), zoom=3)  # Default center of the US

current_county_layer = None  # Store the current county layer

def on_county_change(change):
    global current_county_layer

    selected_county = change['new']
    selected_state = state_dropdown.value

    if current_county_layer is not None:
        # Attempt to remove the previous county layer from the map
        try:
            mymap.remove_layer(current_county_layer)
        except LayerException:
            pass  # If the layer is not on the map, ignore the exception

    if selected_county and selected_state:
        # Filter the GeoDataFrame for the selected state and county
        county_gdf = gdf[(gdf['NAME'] == selected_county) & (gdf['StateName'] == selected_state)]

        if not county_gdf.empty:
            # Convert the county GeoDataFrame to GeoJSON
            county_geojson = json.loads(county_gdf.to_json())

            # Create a GeoJSON layer for the selected county
            county_layer = GeoJSON(data=county_geojson, style={'color': 'blue', 'weight': 2})

            # Add the county layer to the map
            mymap.add_layer(county_layer)

            # Update the global variable to store the current county layer
            current_county_layer = county_layer

            # Calculate the centroid for centering the map view
            centroid = county_gdf.geometry.centroid.iloc[0]
            mymap.center = (centroid.y, centroid.x)
            mymap.zoom = 9  # Adjust the zoom level as needed

county_dropdown.observe(on_county_change, names='value')

# Tool for the toolbox
tool = widgets.VBox([state_dropdown, county_dropdown])

# Create a WidgetControl for the toolbox tool
toolbox_control = WidgetControl(widget=tool, position='topright')

# Add the toolbox control to the map
mymap.add_control(toolbox_control)

C:\Users\Yinan Feng\AppData\Local\Temp\ipykernel_12916\3379553425.py:65: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = county_gdf.geometry.centroid.iloc[0]
C:\Users\Yinan Feng\AppData\Local\Temp\ipykernel_12916\3379553425.py:65: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = county_gdf.geometry.centroid.iloc[0]


In [ ]:

class CountyMap:
    def __init__(self, gdf, state_fips_mapping):
        self.gdf = gdf
        self.state_fips_mapping = state_fips_mapping
        self.gdf['StateName'] = self.gdf['STATEFP'].apply(lambda x: self.state_fips_mapping.get(x, ''))
        self.current_county_layer = None

        self.mymap = Map(center=(37.0902, -95.7129), zoom=3)
        self.setup_widgets()
        self.setup_toolbox()

    def setup_widgets(self):
        # State and County selection
        self.state_dropdown = widgets.Dropdown(description='State:', options=[''] + sorted(self.gdf['StateName'].unique()))
        self.county_dropdown = widgets.Dropdown(description='County:', options=[], disabled=True)
        # Zoom and navigation
        self.zoom_slider = widgets.IntSlider(value=3, min=1, max=20, description='Zoom level:')
        self.lat_input = widgets.FloatText(description='Latitude:', value=37.0902)
        self.lon_input = widgets.FloatText(description='Longitude:', value=-95.7129)
        self.goto_button = widgets.Button(description='Go To')
        self.load_county_map_button = widgets.Button(description='Load All Counties')
        # Set observers and handlers
        self.state_dropdown.observe(self.update_county_dropdown, 'value')
        self.county_dropdown.observe(self.on_county_change, 'value')
        self.goto_button.on_click(self.on_goto_click)
        self.load_county_map_button.on_click(self.on_load_county_map_click)

    def setup_toolbox(self):
        self.search_panel = widgets.VBox([self.zoom_slider, self.lat_input, self.lon_input, self.goto_button])
        self.county_panel = widgets.VBox([self.state_dropdown, self.county_dropdown])
        self.search_button = widgets.Button(description='Search', layout={'width': 'auto'})
        self.county_select_button = widgets.Button(description='County Select', layout={'width': 'auto'})
        self.toolbox_buttons = widgets.VBox([self.search_button, self.county_select_button, self.load_county_map_button])
        self.search_button.on_click(self.toggle_search_panel)
        self.county_select_button.on_click(self.toggle_county_panel)
        # Initial panel setup
        self.panels = widgets.VBox([self.search_panel, self.county_panel])
        self.panels.children[1].layout.display = 'none'  # Hide county panel initially
        # WidgetControl for the map
        self.toolbox_control = WidgetControl(widget=widgets.VBox([self.toolbox_buttons, self.panels]), position='topright')
        self.mymap.add_control(self.toolbox_control)

    def toggle_search_panel(self, b):
        self.panels.children[0].layout.display = ''  # Show search panel
        self.panels.children[1].layout.display = 'none'  # Hide county panel

    def toggle_county_panel(self, b):
        self.panels.children[0].layout.display = 'none'  # Hide search panel
        self.panels.children[1].layout.display = ''  # Show county panel

    def update_county_dropdown(self, change):
        selected_state = self.state_dropdown.value
        filtered_gdf = self.gdf[self.gdf['StateName'] == selected_state]
        self.county_dropdown.options = [''] + sorted(filtered_gdf['NAME'].unique())
        self.county_dropdown.disabled = False

    def on_county_change(self, change):
        selected_county = change['new']
        selected_state = self.state_dropdown.value
        county_gdf = self.gdf[(self.gdf['NAME'] == selected_county) & (self.gdf['StateName'] == selected_state)]
        if not county_gdf.empty:
            county_geojson = json.loads(county_gdf.to_json())
            county_layer = GeoJSON(data=county_geojson, style={'color': 'blue', 'opacity': 0.5, 'weight': 2})
            if self.current_county_layer is not None:
                self.mymap.remove_layer(self.current_county_layer)
            self.mymap.add_layer(county_layer)
            self.current_county_layer = county_layer
            centroid = county_gdf.geometry.centroid.iloc[0]
            self.mymap.center = (centroid.y, centroid.x)
            self.mymap.zoom = 9

    def on_goto_click(self, b):
        self.mymap.center = (self.lat_input.value, self.lon_input.value)
        self.mymap.zoom = self.zoom_slider.value

    def on_load_county_map_click(self, b):
        # Clear all existing layers first, except the base tile layer
        while len(self.mymap.layers) > 1:
            self.mymap.remove_layer(self.mymap.layers[-1])

        # Iterate through each unique county in the GeoDataFrame and add it to the map
        for _, row in self.gdf.iterrows():
            county_geojson = json.loads(row['geometry'].to_json())
            # Construct a GeoJSON layer for each county
            county_layer = GeoJSON(
                data=county_geojson,
                style={'color': 'blue', 'fillColor': '#cccccc', 'weight': 1, 'fillOpacity': 0.5}
            )
            self.mymap.add_layer(county_layer)

In [ ]:
# Usage:
# Assuming 'gdf' is your GeoDataFrame and 'state_fips_mapping' is your FIPS mapping dictionary
county_map = CountyMap(gdf, state_fips_mapping)
county_map.mymap  # This is how you access the map to display it

Map(center=[37.0902, -95.7129], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

AttributeError: 'Polygon' object has no attribute 'to_json'

C:\Users\Yinan Feng\AppData\Local\Temp\ipykernel_12916\2364845946.py:68: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = county_gdf.geometry.centroid.iloc[0]
C:\Users\Yinan Feng\AppData\Local\Temp\ipykernel_12916\2364845946.py:68: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = county_gdf.geometry.centroid.iloc[0]
